# Feature Extraction

In [ ]:
from brainlit.utils.ngl_pipeline import NeuroglancerSession
from brainlit.preprocessing.features import *
import pandas as pd
import numpy as np
import glob

## Feature extraction using brainlit
The classes `LinearFeatures` and `NeighborhoodFeatures` convert swc files to a list of axon/background examples in a dataframe, with relevant features. 

First, instantiate the classes. You pass a `url` to pull data from, a `size` of the bounding box around each point, where the actual box is `2i+1` for each index `i`, and an `offset` to shift the bounding box to get a background point.

In [ ]:
lin = LinearFeatures(url="s3://open-neurodata/brainlit/brain1", size=[1,1,1], offset=[15,15,15])
nbr = NeighborhoodFeatures(url="s3://open-neurodata/brainlit/brain1", size=[1,1,1], offset=[15,15,15])

For the `LinearFeatures` class, you need to define the filters you want to convolve with the neighborhoods. To do so, you use the `add_filters` method. Currently, `brainlit` supports adding Gaussian, Gaussian Gradient Magnitude, Gaussian Laplace, and Gabor filters.

In [ ]:
lin.add_filter('gaussian', sigma=[1, 1, 0.3])
lin.add_filter('gaussian gradient', sigma=[1, 1, 0.3])
lin.add_filter('gaussian laplace', sigma=[1, 1, 0.3])
lin.add_filter('gabor', sigma=[1, 1, 0.3], phi=[0, 0], frequency=2)
# lin.add_filter('gabor', sigma=[1, 1, 0.3], phi=[0, np.pi/2], frequency=2)

Calling the `fit` method for each class with a list of ids and a number of vertices to fit onto for each swc. If the second argument isn't given (or passed as `None`), features will be collected for every vertex.

In [ ]:
df_lin = lin.fit(seg_ids=[2, 7], num_verts=5)
df_nbr = nbr.fit(seg_ids=[2, 7], num_verts=5)

The outputted dataframes themselves have a `Segment` column, `Vertex` column, `Label` column (1=axon 0=background), and feature columns that are indexed starting at 0.

In [ ]:
df_lin

In [ ]:
df_nbr

Both neighborhood and linear features can be extracted using the `include_neighborhood` parameter in the `LinearFeatures` `fit` function.

In [ ]:
df_lin = lin.fit(seg_ids=[2, 7], num_verts=5, include_neighborhood=True)
df_lin

## Saving Extracted Features

In cases where one wishes to extract features from many vertices, `Brainlit` allows batch loading and writing of the data. The data is written into the binary `feather` format.

In [ ]:
df_lin = lin.fit(seg_ids=[2, 7], num_verts=10, file_path='demo', batch_size=10)

Each `batch_size` samples is output as a `feather` file. The `file_path` argument determines the prefix of the filenames. Afterwards, it is followed by the starting sample number, the last sample number, the segment ID for the last sample in the batch, and the vertex ID for the last sample in the batch.

In [ ]:
sorted(glob.glob('*.feather'))

Using the `start_seg` and `start_vert` arguments, you can choose where in the data to start extracting information from.

In [ ]:
df_lin = lin.fit(seg_ids=[2, 7], num_verts=10, file_path='demo_2_', batch_size=10, start_seg=7, start_vert=4)

In [ ]:
sorted(glob.glob('demo_2*.feather'))